In [6]:
import json
import random
import numpy as np
import nltk
import pickle
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

In [7]:
# Download required NLTK data
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package punkt to
[nltk_data]     /home/04cf568e-0075-4f1a-82fa-
[nltk_data]     cbad27b49b0d/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/04cf568e-0075-4f1a-82fa-
[nltk_data]     cbad27b49b0d/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
# Load JSON data
with open('intents.json') as file:
    intents = json.load(file)

In [10]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

In [11]:
import nltk
nltk.download('punkt', force=True)
nltk.download('wordnet', force=True)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/04cf568e-0075-4f1a-82fa-
[nltk_data]     cbad27b49b0d/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/04cf568e-0075-4f1a-82fa-
[nltk_data]     cbad27b49b0d/nltk_data...


True

In [12]:
from nltk.tokenize import word_tokenize
print(word_tokenize("This is a test sentence."))

['This', 'is', 'a', 'test', 'sentence', '.']


In [13]:
# Process patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

In [14]:
# Lemmatize and clean
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [15]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [16]:
# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = [lemmatizer.lemmatize(w.lower()) for w in doc[0]]

    for word in words:
        bag.append(1 if word in pattern_words else 0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [17]:
# Shuffle and convert
random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [18]:
# Build the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


2025-03-14 05:29:43.620605: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-14 05:29:43.625644: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [21]:
# Compile model
# Option 1: Using legacy optimizer (if you want to keep using 'decay')
from tensorflow.keras.optimizers.legacy import SGD

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])



In [22]:
# Train model
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)


Epoch 1/200
3/3 [==============================] - 1s 12ms/step - loss: 1.5769 - accuracy: 0.3571
Epoch 2/200
3/3 [==============================] - 0s 9ms/step - loss: 1.5558 - accuracy: 0.2857
Epoch 3/200
3/3 [==============================] - 0s 7ms/step - loss: 1.6408 - accuracy: 0.1429
Epoch 4/200
3/3 [==============================] - 0s 6ms/step - loss: 1.5341 - accuracy: 0.2143
Epoch 5/200
3/3 [==============================] - 0s 7ms/step - loss: 1.4650 - accuracy: 0.4286
Epoch 6/200
3/3 [==============================] - 0s 19ms/step - loss: 1.4527 - accuracy: 0.2857
Epoch 7/200
3/3 [==============================] - 0s 36ms/step - loss: 1.3369 - accuracy: 0.4286
Epoch 8/200
3/3 [==============================] - 0s 12ms/step - loss: 1.2808 - accuracy: 0.4286
Epoch 9/200
3/3 [==============================] - 0s 28ms/step - loss: 1.2958 - accuracy: 0.3571
Epoch 10/200
3/3 [==============================] - 0s 14ms/step - loss: 1.2707 - accuracy: 0.4286
Epoch 11/200
3/3 [=====

In [ ]:
# Save model
model.save('chatbot_model.h5', include_optimizer=False)
print("Model trained and saved!")


Model trained and saved!
